In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.cluster import AffinityPropagation

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory



# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from matplotlib import pyplot as plt
import seaborn as sns

In [ ]:
timeSeriesConfirmed = pd.read_csv('../input/covid19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv')
timeSeriesConfirmed

In [ ]:
timeSeriesConfirmed = timeSeriesConfirmed.drop(columns = ['Lat', 'Long'])
timeSeriesConfirmed = timeSeriesConfirmed.groupby('Country/Region').sum().reset_index()
timeSeriesConfirmed = timeSeriesConfirmed.rename(columns = {'Country/Region':'Country'})
timeSeriesConfirmed

In [ ]:
timeSeriesDailyNew = timeSeriesConfirmed.copy()
for i in range(len(timeSeriesDailyNew.columns)-2):
    timeSeriesDailyNew.iloc[:,-1-i] = timeSeriesDailyNew.iloc[:,-1-i] - timeSeriesDailyNew.iloc[:,-2-i]
# timeSeriesDailyNew = timeSeriesDailyNew.set_index('Country')
timeSeriesDailyNew

In [ ]:
#timeSeriesDailyNew=timeSeriesDailyNew.astype(float)

#plt.show()
#timeSeriesDailyNew["a"] = timeSeriesDailyNew.to_numeric(df["a"])
plt.figure(figsize=(20,6))
for col in timeSeriesDailyNew.columns:
    if col != 'Country' :
        timeSeriesDailyNew[col] = pd.to_numeric(timeSeriesDailyNew[col])
    
row = timeSeriesDailyNew.iloc[79][2:]
row.plot(kind = 'line')
timeSeriesDailyNew.iloc[79][0:1]

In [ ]:
def toPolyRegression(country, nDays):
    timeSeriesCountry = timeSeriesDailyNew[timeSeriesDailyNew.Country == country]
    daysIntervalnDays = []
    i = 0
    while True:
        try:
            daysIntervalnDays.append((((i)*nDays), np.polyfit(list(np.linspace((i)*nDays,(i+1)*nDays,nDays)), list(timeSeriesCountry.iloc[0][1 + (i*nDays) : nDays + 1 + (i*nDays)].values), 2)))
            i = i + 1
        except:
            break
    return daysIntervalnDays

def toPolyRegressionAbsolute(country, nDays):
    timeSeriesCountry = timeSeriesDailyNew[timeSeriesDailyNew.Country == country]
    daysIntervalnDays = []
    i = 0
    while True:
        try:
            daysIntervalnDays.append((((i)*nDays), np.polyfit(list(np.linspace(1, nDays, nDays)), list(timeSeriesCountry.iloc[0][1 + (i*nDays) : nDays + 1 + (i*nDays)].values), 2)))
            i = i + 1
        except:
            break
    return daysIntervalnDays

In [ ]:
plt.figure(figsize=(30,10))    
plt.plot(list(np.linspace(1,14,14)), list(timeSeriesDailyNew[timeSeriesDailyNew.Country == 'US'].iloc[0][56:70].values))

In [ ]:
plt.figure(figsize=(30,10))    
poly = np.polyfit(list(np.linspace(1,14,14)), list(timeSeriesDailyNew[timeSeriesDailyNew.Country == 'US'].iloc[0][56:70].values), 2)
xp = np.linspace(1,14,200)
plt.plot(list(np.linspace(1,14,14)), list(timeSeriesDailyNew[timeSeriesDailyNew.Country == 'US'].iloc[0][56:70].values))
plt.plot(xp, np.polyval(poly,xp), 'r--')

In [ ]:
plt.figure(figsize=(30,10))
country = 'US'
polyUS = toPolyRegression(country, nDays = 14)
timeSeriesCountry = timeSeriesDailyNew[timeSeriesDailyNew.Country == country]
totalDays = len(timeSeriesCountry.iloc[0][1:].values)
plt.plot(totalDays, totalDays, timeSeriesCountry.iloc[0][1:].values, 'b:')

for day, poly in polyUS:
    xp = np.linspace(day, day+14 ,200)
    plt.plot(xp, np.polyval(poly, xp), 'r-')

In [ ]:
toPolyRegressionAbsolute('US', 14)

In [ ]:
def polyRegressionToAB(poly) :
    AB = []
    for day, curve in poly:
        AB.append(list(curve[:2]))
    return AB

polyRegressionToAB(toPolyRegressionAbsolute('US', 14))


In [ ]:
countries = []
for co in range(0,192) :
    countries.append(timeSeriesDailyNew.iloc[co][0])
countries


In [ ]:
coeff = []

for country in countries :
    temp = polyRegressionToAB(toPolyRegressionAbsolute(country, 14))
    for i in temp :
        coeff.append(i)
    
coeff

In [ ]:
def polyRegressionToAB(poly) :
    AB = []
    for day, curve in poly:
        AB.append(list(curve[:2]))
    return AB

def getCoeffAccum():
    temp = []
    countries = list(timeSeriesDailyNew['Country'])
    for country in countries:
        temp = temp + polyRegressionToAB(toPolyRegressionAbsolute(country, 14))
    return temp


In [ ]:
plt.figure(figsize=(15,5))
plt.xlim([-500, 500])
plt.ylim([-10000, 10000])
x = [_[0] for _ in getCoeffAccum()]
y = [_[1] for _ in getCoeffAccum()]
plt.scatter(x,y)

In [ ]:
from sklearn.cluster import AffinityPropagation

clustering = AffinityPropagation(verbose = True, damping = 0.999, max_iter = 1000, random_state = 0).fit(np.array(getCoeffAccum()))
print('Number of clusters Formed: ', max(clustering.labels_))

In [ ]:
def getClusterLabelsByCountry(counrty):
    return clustering.predict(polyRegressionToAB(toPolyRegressionAbsolute(country, 14)))

getClusterLabelsByCountry('US')

In [ ]:
countries = list(timeSeriesDailyNew['Country'])
trainingClusterLabels = {}
testingClusterLabels = {}

for country in countries:
    trainingClusterLabels[country] = getClusterLabelsByCountry(country)
    testingClusterLabels[country] = getClusterLabelsByCountry(country)[15:25]

testingClusterLabels

In [ ]:
def getIndexNGram(ng, labels):
    indices = []
    for i in range(len(labels) - len(ng)):
        flag = True
        for j in range(len(ng)):
            if ng[j] != labels [j + i]:
                flag = False
        if flag:
            indices.append(i)
    return indices


# getIndexNGram([8,16], [ 70,  8,  16, 224, 163, 163,   8,  16,   2,   8])

def findProbab(ipNGram, trainingSet):
    predFrequency = {}
    total = 0
    for country, labels in trainingSet.items():
        indices = getIndexNGram(ipNGram, labels)
        predLabels = [labels[i+len(ipNGram)] for i in indices]
        for _ in predLabels:
            if _ in predFrequency:
                predFrequency[_] = predFrequency[_] + 1
            else:
                predFrequency[_] = 1
    for _ in predFrequency.values():
        total = total + _
    predProbab = {}
    for a,b in predFrequency.items():
        predProbab[a] = b*100/total
    return predProbab

In [ ]:
findProbab([69,39], trainingClusterLabels)